In [1]:
import os
import itertools
import click
import pandas as pd
from dwave.system import LeapHybridCQMSampler
from dimod import ConstrainedQuadraticModel, DiscreteQuadraticModel, QuadraticModel

In [2]:
def parse_data(data_file):
    df = pd.read_csv(data_file, names=['food', 'type', 'serving', 'calories', 'protein', 'carbs', 'fat', 'price'])
    return df['food'], df['calories'], df['protein'], df['carbs'], df['fat'], df['price'], df['serving']

In [3]:
def build_knapsack_cqm(prices, calories, protein, carbs, fats, max_cals, max_prot, max_carbs, max_fats):
    num_items = len(calories)

    cqm = ConstrainedQuadraticModel()
    obj = QuadraticModel()
    constraint_cals = QuadraticModel()
    constraint_prot = QuadraticModel()
    constraint_carbs = QuadraticModel()
    constraint_fats = QuadraticModel()

    for i in range(num_items):
        # Objective is to maximize the total costs
        obj.add_variable('INTEGER', i, upper_bound = 3)
        obj.set_linear(i, prices[i])
        # Constraint is to keep the sum of items' weights under or equal capacity
        constraint_cals.add_variable('INTEGER', i, upper_bound = 3)
        constraint_cals.set_linear(i, calories[i])
        constraint_prot.add_variable('INTEGER', i, upper_bound = 3)
        constraint_prot.set_linear(i, protein[i])
        constraint_carbs.add_variable('INTEGER', upper_bound = 3)
        constraint_carbs.set_linear(i, carbs[i])
        constraint_fats.add_variable('INTEGER', upper_bound = 3)
        constraint_fats.set_linear(i, fats[i])

    cqm.set_objective(obj)
    cqm.add_constraint(constraint_cals, sense=">=", rhs=max_cals, label='low_calories')
    cqm.add_constraint(constraint_cals, sense="<=", rhs=max_cals*1.05, label='up_calories')
    cqm.add_constraint(constraint_prot, sense=">=", rhs=max_prot, label='low_protein')
    cqm.add_constraint(constraint_prot, sense="<=", rhs=max_prot*1.05, label='up_protein')
    cqm.add_constraint(constraint_carbs, sense=">=", rhs=max_carbs, label='low_carbs')
    cqm.add_constraint(constraint_carbs, sense="<=", rhs=max_carbs*1.05, label='up_carbs')
    cqm.add_constraint(constraint_fats, sense=">=", rhs=max_fats, label='low_fats')
    cqm.add_constraint(constraint_fats, sense="<=", rhs=max_fats*1.05, label='up_fats')

    return cqm

In [6]:
#sampler = LeapHybridCQMSampler(token="DEV-081a4f10459d3e62cd6fbb680e6be1cd935abefb")
max_cals = 2000
max_prot = (max_cals*0.3)/4
max_carbs = (max_cals*0.5)/4
max_fats = (max_cals*0.2)/9
foods, calories, protein, carbs, fats, prices, serving = parse_data("./foods/data.csv")
#cqm = build_knapsack_cqm(prices, calories, protein, carbs, fats, max_cals, max_prot, max_carbs, max_fats)
#sampleset = sampler.sample_cqm(cqm, label='Example - Diet Opt')

In [95]:
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
best = feasible_sampleset.first
print(best)

Sample(sample={0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 3.0, 16: 0.0, 17: 1.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 3.0, 22: 0.0, 23: 3.0, 24: 0.0, 25: 0.0, 26: 1.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 0.0, 44: 0.0, 45: 0.0, 46: 0.0, 47: 3.0, 48: 0.0, 49: 0.0, 50: 0.0, 51: 0.0, 52: 0.0, 53: 2.0, 54: 0.0, 55: 1.0, 56: 0.0, 57: 0.0, 58: 0.0, 59: 0.0, 60: 0.0, 61: 0.0, 62: 1.0, 63: 0.0}, energy=10.08, num_occurrences=1, is_feasible=True, is_satisfied=array([ True,  True,  True,  True,  True,  True,  True,  True]))


In [96]:
selected_item_indices = [key for key, val in best.sample.items() if val>0.0]
selected_item_amounts = [val for key, val in best.sample.items() if val>0.0]
for i in range(len(selected_item_indices)):
    index = selected_item_indices[i]
    amount = selected_item_amounts[i]
    print(foods[index] + " (" + serving[index] + ") " + str(amount))
selected_cals = list(calories.loc[selected_item_indices])
selected_prot = list(protein.loc[selected_item_indices])
selected_carbs = list(carbs.loc[selected_item_indices])
selected_fats = list(fats.loc[selected_item_indices])
total_cal = 0
total_prot = 0
total_carbs = 0
total_fats = 0
for i in range(len(selected_cals)):
    total_cal += selected_cals[i] * selected_item_amounts[i]
    total_prot += selected_prot[i] * selected_item_amounts[i]
    total_carbs += selected_carbs[i] * selected_item_amounts[i]
    total_fats += selected_fats[i] * selected_item_amounts[i]
print(f"Cost per day: {best.energy}")
print(f"Calories: {total_cal}")
print(f"Protein: {total_prot}")
print(f"Carbs: {total_carbs}")
print(f"Fats: {total_fats}")

lentils (1/2cup) 3.0
Bread (1slice) 1.0
Oats (1/2cup) 3.0
Peas-black-eye (1/2cup) 3.0
Tortillas (1tortilla) 1.0
Peanut-Butter (1tbsp) 3.0
Cottage-Cheese-(1%-fat) (1/2cup) 2.0
Egg-Whites (3tbsp) 1.0
Steak-(Strip) (4oz) 1.0
Cost per day: 10.08
Calories: 2036.0
Protein: 150.0
Carbs: 250.0
Fats: 46.0


In [3]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit import BasicAer
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)

In [4]:
def build_knapsack_qp(prices, calories, protein, carbs, fats, max_cals, max_prot, max_carbs, max_fats):
    mod = QuadraticProgram("knapsack")
    num_items = len(prices)
    linear_terms = []
    const_cals = {}
    const_prot = {}
    const_carbs = {}
    const_fats = {}
    for i in range(num_items):
        mod.integer_var(upperbound=3, name="x_"+str(i))
        linear_terms.append(prices[i])
        const_cals["x_"+str(i)] = calories[i]
        const_prot["x_"+str(i)] = protein[i]
        const_carbs["x_"+str(i)] = carbs[i]
        const_fats["x_"+str(i)] = fats[i]

    mod.linear_constraint(linear=const_cals, sense=">=", rhs=max_cals, name="low_calories")
    mod.linear_constraint(linear=const_cals, sense="<=", rhs=max_cals*1.05, name="up_calories")
    #mod.linear_constraint(linear=const_prot, sense=">=", rhs=max_prot, name="low_protein")
    #mod.linear_constraint(linear=const_prot, sense="<=", rhs=max_prot*1.05, name="up_protein")
    #mod.linear_constraint(linear=const_carbs, sense=">=", rhs=max_carbs, name="low_carbs")
    #mod.linear_constraint(linear=const_carbs, sense="<=", rhs=max_carbs*1.05, name="up_carbs")
    #mod.linear_constraint(linear=const_fats, sense=">=", rhs=max_fats, name="low_fats")
    #mod.linear_constraint(linear=const_fats, sense="<=", rhs=max_fats*1.05, name="up_fats")
    mod.minimize(linear=linear_terms)

    return mod

In [7]:
test_indices = [54, 18, 47, 21, 0]
test_prices = []
test_calories = []
test_protein = []
test_carbs = []
test_fats = []

for i in test_indices:
    test_prices.append(prices[i])
    test_calories.append(calories[i])
    test_protein.append(protein[i])
    test_carbs.append(carbs[i])
    test_fats.append(fats[i])
mod = build_knapsack_qp(test_prices, test_calories, test_protein, test_carbs, test_fats, max_cals, max_prot, max_carbs, max_fats)
print(mod)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: knapsack

Minimize
 obj: 0.710000000000 x_0 + 0.570000000000 x_1 + 0.120000000000 x_2
      + 0.160000000000 x_3 + 1.520000000000 x_4
Subject To
 low_calories: 80 x_0 + 130 x_1 + 94 x_2 + 150 x_3 + 95 x_4 >= 2000
 up_calories: 80 x_0 + 130 x_1 + 94 x_2 + 150 x_3 + 95 x_4 <= 2100

Bounds
       x_0 <= 3
       x_1 <= 3
       x_2 <= 3
       x_3 <= 3
       x_4 <= 3

Generals
 x_0 x_1 x_2 x_3 x_4
End



In [8]:
conv = QuadraticProgramToQubo()
qubo = conv.convert(mod)
op, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: 19915109.899999995
operator:
122926.07999999996 * ZIIIIIIIIIIIIIIIIIIIII
+ 2375024.640000003 * IZIIIIIIIIIIIIIIIIIIII
+ 277872.6400000001 * ZZIIIIIIIIIIIIIIIIIIII
+ 1187512.320000001 * IIZIIIIIIIIIIIIIIIIIII
+ 138936.32000000004 * ZIZIIIIIIIIIIIIIIIIIII
+ 2684354.5600000005 * IZZIIIIIIIIIIIIIIIIIII
+ 593756.1600000003 * IIIZIIIIIIIIIIIIIIIIII
+ 69468.16000000002 * ZIIZIIIIIIIIIIIIIIIIII
+ 1342177.2800000003 * IZIZIIIIIIIIIIIIIIIIII
+ 671088.6400000001 * IIZZIIIIIIIIIIIIIIIIII
+ 296878.08000000037 * IIIIZIIIIIIIIIIIIIIIII
+ 34734.08000000001 * ZIIIZIIIIIIIIIIIIIIIII
+ 671088.6400000001 * IZIIZIIIIIIIIIIIIIIIII
+ 335544.32000000007 * IIZIZIIIIIIIIIIIIIIIII
+ 167772.16000000003 * IIIZZIIIIIIIIIIIIIIIII
+ 148439.04000000018 * IIIIIZIIIIIIIIIIIIIIII
+ 17367.040000000005 * ZIIIIZIIIIIIIIIIIIIIII
+ 335544.32000000007 * IZIIIZIIIIIIIIIIIIIIII
+ 167772.16000000003 * IIZIIZIIIIIIIIIIIIIIII
+ 83886.08000000002 * IIIZIZIIIIIIIIIIIIIIII
+ 41943.04000000001 * IIIIZZIIIIIIIIIIIIIIII
+ 74219.5

In [9]:
print(qubo)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: knapsack

Minimize
 obj: - 6717439.290000000969 x_0@0 - 13434878.580000001937 x_0@1
      - 10915839.430000001565 x_1@0 - 21831678.860000003129 x_1@1
      - 7892991.880000000820 x_2@0 - 15785983.760000001639 x_2@1
      - 12595199.840000001714 x_3@0 - 25190399.680000003427 x_3@1
      - 7976958.480000002310 x_4@0 - 15953916.960000004619 x_4@1
      - 43008.000000000007 up_calories@int_slack@0
      - 86016.000000000015 up_calories@int_slack@1
      - 172032.000000000029 up_calories@int_slack@2
      - 344064.000000000058 up_calories@int_slack@3
      - 688128.000000000116 up_calories@int_slack@4
      - 1376256.000000000233 up_calories@int_slack@5
      - 2752512.000000000466 up_calories@int_slack@6
      - 5505024.000000000931 up_calories@int_slack@7
      - 11010048.000000001863 up_calories@int_slack@8
      - 22020096.000000003725 up_calories@int_slack@9
      - 44040192.000000007451 up_calories@int_slac

In [14]:
algorithm_globals.random_seed = 10598
algorithm_globals.massive=True
quantum_instance = QuantumInstance(
    BasicAer.get_backend("qasm_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0.0, 0.0])
qaoa = MinimumEigenOptimizer(qaoa_mes)
qaoa_result = qaoa.solve(qubo)

In [24]:
print(qaoa_result)

optimal function value: 1312869.400000006
optimal value: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1.]
status: SUCCESS


In [25]:
print([var.name for var in qaoa_result.variables])

['x_0@0', 'x_0@1', 'x_1@0', 'x_1@1', 'x_2@0', 'x_2@1', 'x_3@0', 'x_3@1', 'x_4@0', 'x_4@1', 'up_calories@int_slack@0', 'up_calories@int_slack@1', 'up_calories@int_slack@2', 'up_calories@int_slack@3', 'up_calories@int_slack@4', 'up_calories@int_slack@5', 'up_calories@int_slack@6', 'up_calories@int_slack@7', 'up_calories@int_slack@8', 'up_calories@int_slack@9', 'up_calories@int_slack@10', 'up_calories@int_slack@11']


In [26]:
print(qaoa_result.samples[0])

SolutionSample(x=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1.,
       1., 1., 0., 0., 1.]), fval=1312869.400000006, probability=0.0009765625, status=<OptimizationResultStatus.SUCCESS: 0>)


In [25]:
import csv
prefix = "./foods/"
files = ["carbs-fruits", "carbs-grains", "carbs-vegetables", "fats", "proteins"]
postfix = ".txt"
headers = "Food,Type,Serving,Calories,Protein,Carbs,Fat,Price\n"
data_list = []
# output += headers
f_w = open(prefix+"data.csv", 'w')
writer = csv.writer(f_w)
for file in files:
    full = prefix + file + postfix
    f = open(full, "r")
    for x in f:
        x = x.strip()
        data = x.split(" ")
        line = [data[0],file,data[1]+data[2],data[3],data[4],data[5],data[6],data[7]]
        data_list.append(line)
writer.writerows(data_list)
f_w.close()
